In [1]:
import pandas as pd
from datetime import datetime
from difflib import SequenceMatcher
import numpy as np
from fuzzywuzzy import fuzz

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# Define the file path relative to the current notebook's location
file_path = "../data/complaints 5.9.2023.csv"

# Read the CSV file into a DataFrame
cfpb_df = pd.read_csv(file_path)

C:\Users\JYM\AppData\Local\Temp/ipykernel_44880/2892497937.py:5: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  cfpb_df = pd.read_csv(file_path)


In [4]:
cfpb_df.sample(5)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
2777849,2022-12-24,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,"EQUIFAX, INC.",FL,32757,NaN,Consent not provided,Web,2022-12-24,Closed with explanation,Yes,NaN,6356122
799284,2019-09-27,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,THE DISPUTED ACCOUNTS HAS BEEN HINDERING MY CR...,NaN,"EQUIFAX, INC.",FL,33444.0,NaN,Consent provided,Web,2019-09-27,Closed with explanation,Yes,NaN,3387881
1805569,2020-12-15,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Investigation took more than 30 days,I have sent letters to the credit bureaus beca...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,33183.0,NaN,Consent provided,Web,2020-12-15,Closed with explanation,Yes,NaN,4012531
1512234,2016-02-23,Student loan,Non-federal student loan,Dealing with my lender or servicer,Trouble with how payments are handled,NaN,NaN,"Navient Solutions, LLC.",OR,97223.0,NaN,NaN,Referral,2016-02-29,Closed with explanation,Yes,Yes,1801131
2899367,2016-12-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,These people ( ERC ) calls me at all kinds of ...,Company believes it acted appropriately as aut...,ERC,NC,28306,NaN,Consent provided,Web,2016-12-08,Closed,Yes,No,2241757


In [5]:
%%time
# Convert the 'date' column to datetime format
print("Before droping nan narrative: ", len(cfpb_df))
cfpb_df.dropna(subset=['Consumer complaint narrative'], inplace=True)
cfpb_df['Date received'] = pd.to_datetime(cfpb_df['Date received'])
cfpb_df['narr_len'] = cfpb_df['Consumer complaint narrative'].apply(lambda x:len(str(x)))
cfpb_df['days_to_today'] = (datetime.now().date() - cfpb_df['Date received'].dt.date).dt.days

cfpb_df['narr_len'] = cfpb_df['narr_len'].astype(int)
cfpb_df['days_to_today'] = cfpb_df['days_to_today'].astype(int)


cfpb_df['narr_head'] = cfpb_df['Consumer complaint narrative'].apply(lambda x: x[:200])
print("After droping nan narrative: ", len(cfpb_df))

Before droping nan narrative:  3599657
After droping nan narrative:  1300361
Wall time: 3.93 s


In [6]:
small_cfpb_df = cfpb_df[['Product', 'Issue', 'State', 'ZIP code','Complaint ID','narr_len', 'days_to_today', 'narr_head']].copy()
small_cfpb_df[['Product', 'Issue', 'State', 'ZIP code']] = small_cfpb_df[['Product', 'Issue', 'State', 'ZIP code']].fillna('')

In [7]:
# when running group by, the input is a mini dataframe with same ['Product', 'Issue', 'State', 'ZIP code']
def find_duplicate_narr(df):
    small_df = df[['Complaint ID','narr_len', 'days_to_today', 'narr_head']].copy()
    
    def find_dupi_in_small_df(row_narr_len, row_to_day, row_short_narr, small_df):
#         tmp_df = small_df.query("'narr_len'<=%s & 'narr_len'>=%s & 'days_to_today'<= %s & 'days_to_today'>= %s" % (int(row_narr_len*1.2),
#                                                                                                                    int(row_narr_len*0.8),
#                                                                                                                    int(row_to_day+15),
#                                                                                                                    int(row_to_day-15))).copy()
        tmp_df = small_df.query("narr_len <= @row_narr_len*1.2 & narr_len >= @row_narr_len*0.8 & days_to_today <= @row_to_day+15 & days_to_today >= @row_to_day-15").copy()    
        # tmp_df['similarity_to_row'] = tmp_df['narr_head'].apply(lambda x: SequenceMatcher(None, x, row_short_narr).ratio())
        tmp_df['similarity_to_row'] = tmp_df['narr_head'].apply(lambda x: fuzz.WRatio(x, row_short_narr)/100)
        
        dupli_df = tmp_df[tmp_df['similarity_to_row']>0.85]
        dupli_id_list = sorted(dupli_df['Complaint ID'].to_list())
#         if len(dupli_id_list)>1:
#             print(dupli_id_list)
        return dupli_id_list
    
    df['dupi_id'] = small_df.apply(lambda row: find_dupi_in_small_df(row['narr_len'], int(row['days_to_today']), row['narr_head'], small_df),axis=1)
    
    return df

In [8]:
%%time
small_cfpb_df = small_cfpb_df.groupby(['Product', 'Issue', 'State', 'ZIP code']).apply(func=find_duplicate_narr)

Wall time: 59min 37s


In [9]:
small_cfpb_df['dupi_len'] = small_cfpb_df['dupi_id'].apply(lambda x: len(x))
small_cfpb_df['dupi_id'] = small_cfpb_df['dupi_id'].apply(lambda x: ";".join([str(y) for y in x]))

In [10]:
merged_df = cfpb_df.merge(small_cfpb_df[['Complaint ID', 'dupi_id', 'dupi_len']], on='Complaint ID', how='left').drop(['narr_head'], axis=1)

In [11]:
merged_df.to_csv('CFPB with Duplicate Marked.csv')
merged_df.to_csv('../data/CFPB with Duplicate Marked fuzz.csv')

In [13]:
merged_df[merged_df.dupi_len>1]

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,narr_len,days_to_today,dupi_id,dupi_len
10,2023-04-17,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,I have not supplied proof under the doctrine o...,Company believes the complaint is the result o...,"CCS Financial Services, Inc.",MO,64138.0,...,Web,2023-04-17,Closed with explanation,Yes,NaN,6852167,3444,22,6852167;6852350,2
18,2023-04-19,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,On XX/XX/2023 I sent a notarized affidavit to ...,Company has responded to the consumer and the ...,U.S. BANCORP,TX,77581.0,...,Web,2023-04-19,Closed with non-monetary relief,Yes,NaN,6862521,544,20,6862361;6862521,2
44,2023-04-18,Checking or savings account,Checking account,Managing an account,Problem using a debit or ATM card,"Dear Wells Fargo Bank, Thank you for your resp...",Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,UT,84601.0,...,Web,2023-04-18,Closed with explanation,Yes,NaN,6857381,2443,21,6807705;6857381,2
97,2023-03-11,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,I just received a copy of my credit report and...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TX,75134.0,...,Web,2023-03-11,Closed with non-monetary relief,Yes,NaN,6674343,3213,59,6674336;6674343,2
134,2023-04-11,Debt collection,Credit card debt,Attempts to collect debt not owed,Debt was result of identity theft,I CHECKED MY CREDIT REPORT AND THIS CREDIT LOA...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",NY,10033.0,...,Web,2023-04-11,Closed with non-monetary relief,Yes,NaN,6825305,87,28,6825305;6825630,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300255,2016-09-15,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,I am a veteran widow whom is a recipient of Ma...,NaN,"Receivables Outsourcing, Inc.",MD,20769.0,...,Web,2016-09-15,Closed with explanation,Yes,Yes,2112799,176,2427,2110819;2110923;2112799,3
1300263,2015-08-04,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,THIS COMANY IS REPORTING A NEGATIVE REPORT ON ...,NaN,SYNCHRONY FINANCIAL,GA,30066.0,...,Web,2015-08-04,Closed with explanation,Yes,No,1500708,118,2835,1500704;1500708,2
1300268,2015-12-04,Credit reporting,NaN,Incorrect information on credit report,Personal information,"I have never been In Kansas, Oklahoma ; I was ...",NaN,"EQUIFAX, INC.",TX,75083.0,...,Web,2015-12-04,Closed with explanation,Yes,No,1682881,127,2713,1677253;1682881,2
1300269,2015-03-19,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,"Identity was stolen, and inaccurate informatio...",NaN,"EQUIFAX, INC.",SC,29045.0,...,Web,2015-03-23,Closed with explanation,Yes,No,1291223,86,2973,1291223;1291224,2


In [14]:
merged_df.shape

(1300361, 22)

In [15]:
349816/1300361

0.26901452750428534